## MATH 440 Least-Squares Monte Carlo (LSM)  
### Feb 22, 2025

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

In [3]:
def paths_mc_gbm(S_0,r,sigma,T,N,M):
    dt=T/N
    paths=np.zeros((M,N+1))#with each row denotes different paths and cols denote time series
    paths[:,0]=S_0
    for t in range(1,N+1):
        Z=np.random.normal(0, 1, M)
        paths[:,t]=paths[:,t-1]*np.exp((r-0.5*sigma**2)*dt+sigma*np.sqrt(dt)*Z)
    return paths

In [4]:
def lsm_fit_model(S_0,K,r,sigma,T,N,M,degree):
    dt=T/N
    paths=paths_mc_gbm(S_0,r,sigma,T,N,M)
    V=np.zeros_like(paths)
    V[:,-1]=np.maximum(K-paths[:,-1],0)
    exercise=np.zeros_like(paths, dtype=bool)
    exercise[:,-1]=V[:,-1]>0
    
    for t in range(N-1,0,-1):
        in_the_money=paths[:,t]<K #in-money filter
        X=paths[in_the_money,t].reshape(-1,1)
        
        poly=PolynomialFeatures(degree)
        X_poly=poly.fit_transform(X)
        Y=V[in_the_money,t+1]*np.exp(-r*dt)
        model=LinearRegression(fit_intercept=False)
        model.fit(X_poly,Y)
        continuation=model.predict(X_poly)
        exercise_value=K-X.flatten()
        exercise[in_the_money,t]=exercise_value>continuation
        V[in_the_money,t]=np.where(exercise[in_the_money,t],exercise_value,Y)
        V[~in_the_money,t]=V[~in_the_money,t+1]*np.exp(-r*dt)

    first_exercise=np.argmax(exercise,axis=1)
    payoff_times=np.where(exercise.any(axis=1),first_exercise,N)
    payoffs=V[np.arange(M), payoff_times]  
    return np.mean(payoffs*np.exp(-r*payoff_times*dt))

In [5]:
S_0=100
K=100
r=0.05
sigma=0.2
T=1 #time in total
N=50 #number of steps
M=10000 #number of paths

option_value=lsm_fit_model(S_0,K,r,sigma,T,N,M,degree=3)
print(option_value)

6.125723333684576
